# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [114]:
import sqlite3
conn = sqlite3.connect('../../assets/datasets/enron.db')
c = conn.cursor()

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [115]:
fields = c.execute("SELECT sql FROM sqlite_master where type='table' and name = 'EmployeeBase';").fetchall()
print''.join(fields[0])

fields = c.execute("SELECT sql FROM sqlite_master where type='table' and name = 'MessageBase';").fetchall()
print''.join(fields[0])

fields = c.execute("SELECT sql FROM sqlite_master where type='table' and name = 'RecipientBase';").fetchall()
print''.join(fields[0])

CREATE TABLE EmployeeBase (
                  [eid] INTEGER,
  [name] TEXT,
  [department] TEXT,
  [longdepartment] TEXT,
  [title] TEXT,
  [gender] TEXT,
  [seniority] TEXT
                  
                  )
CREATE TABLE MessageBase (
    mid INTEGER,
    filename TEXT,
    unix_time INTEGER,
    subject TEXT,
    from_eid INTEGER,
    
    PRIMARY KEY(mid ASC),
    FOREIGN KEY(from_eid) REFERENCES Employee(eid)
)
CREATE TABLE RecipientBase (
    mid INTEGER,
    rno INTEGER,
    to_eid INTEGER,
    
    PRIMARY KEY(mid ASC, rno ASC)
    FOREIGN KEY(mid) REFERENCES Message(mid)
    FOREIGN KEY(to_eid) REFERENCES Employee(eid)
)


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [116]:
results1 = c.execute("SELECT * FROM EmployeeBase LIMIT 5").fetchall()
for row in results1:
    print row


results2 = c.execute("SELECT * FROM MessageBase LIMIT 5").fetchall()
for row in results2:
    print row

results3 = c.execute("SELECT * FROM RecipientBase LIMIT 5").fetchall()
for row in results3:
    print row

(1, u'John Arnold', u'Forestry', u'ENA Gas Financial', u'VP Trading', u'Male', u'Senior')
(2, u'Harry Arora', u'Forestry', u'ENA East Power', u'VP Trading', u'Male', u'Senior')
(3, u'Robert Badeer', u'Forestry', u'ENA West Power', u'Mgr Trading', u'Male', u'Junior')
(4, u'Susan Bailey', u'Legal', u'ENA Legal', u'Specialist Legal', u'Female', u'Junior')
(5, u'Eric Bass', u'Forestry', u'ENA Gas Texas', u'Trader', u'Male', u'Junior')
(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138)
(2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138)
(3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138)
(4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138)
(5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)
(1, 1, 59)
(2, 1, 15)
(3, 1, 15)
(4, 1, 109)
(4, 2, 49)


Import each of the 3 tables to a Pandas Dataframes

In [117]:
import pandas as pd
from pandas.io import sql

EmployeeBase = sql.read_sql('select * from EmployeeBase', con=conn)

MessageBase = sql.read_sql('select * from MessageBase', con=conn)

RecipientBase = sql.read_sql('select * from RecipientBase', con=conn)


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [118]:
print len(EmployeeBase)
print len(MessageBase)
MessageBase.head(20)
import time
import datetime
MessageBase['unix_time'] = MessageBase['unix_time'].astype(int)

a = []
for i in (MessageBase['unix_time']):
     a.append(datetime.datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S'))
    
MessageBase['unix_time'] = a
MessageBase.sort_values(by='unix_time')



156
21635


,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,1998-11-12 23:07:00,Cd$ CME letter,138
1,2,taylor-m/sent/17,1998-11-19 02:19:00,Indemnification,138
2,3,taylor-m/sent/18,1998-11-19 03:24:00,Re: Indemnification,138
3,4,taylor-m/sent/23,1998-11-23 21:23:00,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,1998-11-29 22:22:00,Bankruptcy Code revisions,138
5,6,taylor-m/sent/31,1998-12-01 22:47:00,Re: Position Description,138
6,7,taylor-m/sent/33,1998-12-02 00:24:00,Koch,138
7,8,taylor-m/sent/40,1998-12-03 06:38:00,Re: Time to Celebrate!,138
8,9,taylor-m/sent/41,1998-12-03 20:15:00,Re: Vacation Request,138
9,10,taylor-m/sent/44,1998-12-08 20:14:00,Re: Last Message,138


Rescale to investigate the tail of the curve

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.